In [2]:
import glob
import os
import shutil
#import tempfile
import yaml
from tqdm import tqdm
import pandas as pd

import nibabel as nib
import numpy as np
import torch
import matplotlib.pyplot as plt

import monai
from monai.transforms import SaveImage

In [3]:
with open('./config.yaml', 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

x_image_size = cfg['data_preparation']['image_size_x']
y_image_size = cfg['data_preparation']['image_size_y']
z_image_size = cfg['data_preparation']['image_size_z']
DICOM_FOLDER = cfg['preprocessing']['root_dicom_folder']
root_dataFolder = os.path.split(DICOM_FOLDER)[0]
NIFTI_direc = os.path.join(root_dataFolder,'NIFTI_test')
NIFIT_slices_direc = os.path.join(root_dataFolder,'NIFTI_SCLICES_test')


FileNotFoundError: [Errno 2] No such file or directory: './config.yaml'

In [ ]:
# filepaths = []
# filenames = []
# for root, dirs, files in os.walk(NIFTI_direc):
#     for i_file in files:
#         if '.nii.gz' in i_file:
#             filepaths.append(os.path.join(root,i_file))
#             filenames.append(i_file.split('.nii.gz')[0])
# print(len(filepaths))
resultFrame = pd.read_csv('.testing/out/Predictions_DDS_model_epochs100_time_2023-09-21_16:16:19.587076/Predictions_DDS_model_epochs100_time_2023-09-21_16:16_ensamblePredictions.csv')
missclassifications = resultFrame[resultFrame['groundTruth']!=resultFrame['vote']]
missclassifications = missclassifications[missclassifications['certainty']==1]

In [ ]:
T1examples = resultFrame[resultFrame['groundTruth']==1]

In [ ]:
dataset = [{'image':filepath,'label':filename} for filepath, filename in zip(T1examples.imageID,T1examples.NIFTI_name)]

In [ ]:
dataset = [{'image':filepath,'label':filename} for filepath, filename in zip(missclassifications.imageID,missclassifications.NIFTI_name)]

In [ ]:
imgIDX = 2
img_path = dataset[imgIDX]['image']
nimg = nib.load(img_path)
data = nimg.get_fdata()
affine = nimg.affine
header = nimg.header
print(data.shape,data.dtype,'\n',affine.shape,affine.dtype)
plt.figure()
plt.imshow(data[0,:,:],cmap='gray')
plt.title(dataset[imgIDX]['label'])
plt.show()

In [ ]:
class SaveIndividualSlices(monai.transforms.MapTransform):
    def __init__(self, keys, targetDir):
        self.keys = keys
        self.targetDir = targetDir

    def __call__(self, data):
        os.makedirs(self.targetDir,exist_ok=True)
        depth = data[self.keys[0]].shape[-1]
        for i in range(depth):
            if i<10:
                postfix = f'_s0{i}'
            else:
                postfix = f'_s{i}'
            saver = SaveImage(
                output_dir=self.targetDir,
                output_postfix=postfix,
                output_ext='.nii.gz',
                resample=False,
                squeeze_end_dims=True,
                data_root_dir=NIFTI_direc,
                separate_folder=False,
                print_log=False,
                savepath_in_metadict=True,
            )
            slice = data[self.keys[0]][:,:,:,i]
            slice = np.array(slice)
            slice = slice.reshape((1,256,256,1))
            metadata = dict(data[f'{self.keys[0]}_meta_dict'])
            saver(img=slice,meta_data=metadata)
            # filename = str(data[self.keys[1]])
            # savepath = os.path.join(self.targetDir,f'{filename}__s{i}.nii.gz')
            # affine = metadata['affine']
            # nib.save(img=nib.Nifti1Pair(slice,affine=affine), filename=savepath)
        return data

In [ ]:
baseTransform = monai.transforms.Compose(
    [
        monai.transforms.LoadImaged(keys=['image'], image_only=False, ensure_channel_first=False),
        monai.transforms.EnsureChannelFirstd(keys=['image']),
        monai.transforms.Orientationd(keys=['image'],axcodes='RAS'),
        monai.transforms.Resized(keys=['image'], spatial_size=[256, 256, 25]),
        monai.transforms.ScaleIntensityd(keys=['image']),
        SaveIndividualSlices(keys=['image', 'label'], targetDir=NIFIT_slices_direc)
    ]
)

In [ ]:
ds_2d = monai.data.Dataset(dataset, baseTransform)
dl_2d = monai.data.DataLoader(ds_2d, batch_size=1, num_workers=0,shuffle=False)

In [ ]:
counter = -1
try:
    for i in tqdm(dl_2d):
        counter+=1
        continue
except:
    print(i['label'])

In [ ]:
filepaths2 = []
filenames2 = []
dataset2 = []
for root, dirs, files in os.walk(NIFIT_slices_direc,topdown=True):
    for i_file in sorted(files):
        if '.nii.gz' in i_file:
            filepaths2.append(os.path.join(root,i_file))
            filenames2.append(i_file.split('.nii.gz')[0])
print(len(filepaths2))
dataset2 = [{'image':filepath,'label':filename} for filepath, filename in zip(filepaths2,filenames2)]

In [ ]:
imgIDX = 1
img_path = dataset2[imgIDX]['image']
nimg = nib.load(img_path)
data = nimg.get_fdata()
affine = nimg.affine
header = nimg.header
print(data.shape,data.dtype,'\n',affine.shape,affine.dtype)
plt.figure()
plt.imshow(data[:,:],cmap='gray')
plt.title(dataset2[imgIDX]['label'])
plt.show()

In [ ]:
plt.hist((data[:,:].flatten()))